In [193]:
from chembl_webresource_client.new_client import new_client
from chembl_webresource_client.query_set import QuerySet
import pandas as pd
from os import mkdir

try:
  mkdir("results")
  mkdir("results/compounds")
  mkdir("results/compounds/molfiles")
except:
  pass

In [194]:
# type: ignore

def DataFrameMolfilesFromIdList(molecule_chembl_id_list: list[str]) -> pd.DataFrame:
    data: QuerySet = new_client.molecule.filter(molecule_chembl_id__in=molecule_chembl_id_list).only(['molecule_chembl_id', 'molecule_structures'])
    df = pd.DataFrame(data)

    df['molfile'] = df['molecule_structures'].apply(lambda x: x['molfile'] if isinstance(x, dict) else None)

    df = df.drop(['molecule_structures'], axis=1)

    return df

In [195]:
df = DataFrameMolfilesFromIdList(["CHEMBL266160", "CHEMBL14449", "CHEMBL14688"])
df

,molecule_chembl_id,molfile
0,CHEMBL266160,\n RDKit 2D\n\n 3 2 0 0 0 0...
1,CHEMBL14449,\n RDKit 2D\n\n 3 2 0 0 0 0...
2,CHEMBL14688,\n RDKit 2D\n\n 2 1 0 0 0 0...


In [197]:
def SaveMolfilesToSDF(df: pd.DataFrame, file_name: str):
    with open(file_name, 'w') as f:
        for value in df.values:
            molecule_chembl_id, molfile = value
            f.write(f"{molecule_chembl_id}{molfile}\n\n$$$$\n")

In [198]:
SaveMolfilesToSDF(df, "results/compounds/molfiles/molfiles.sdf")